In [41]:
import pickle
import os
import pandas as pd
import numpy as np


In [3]:
model_path = r"C:\Users\HP\Desktop\seoul-bike-sharing-prediction\models\xgboost_regressor_r2_0_928_v1.pkl"
model = pickle.load(open(model_path,"rb"))

In [5]:
# load scaling file
sc_dump_path = r"C:\Users\HP\Desktop\seoul-bike-sharing-prediction\models\sc.pkl"
sc = pickle.load(open(sc_dump_path,"rb"))

In [6]:
sc

StandardScaler()

In [119]:
# users input
date="10/06/2023"
hour = 21
temperature = 34
humidity=85
wind_speed=67
visibility=100
solar_radation=0.0
rainfall = 0.7
snowfall=0.2 
seasons="Summer"
holiday="No Holiday"
functioning_day = "Yes"


#converting user data into model consumable format

In [120]:
from datetime import datetime
holiday_dic = {"No Holiday":0,"Holiday":1}
finctioning_day = {"No":0,"Yes":1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date,'%d/%m/%Y')
    return {"day":dt.day,"month":dt.month,"year":dt.year,"weekday":dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)
str_to_date
    
    




{'day': 10, 'month': 6, 'year': 2023, 'weekday': 'Saturday'}

In [121]:
u_input_list = [hour,temperature,humidity,wind_speed,visibility,solar_radation,rainfall,snowfall,holiday_dic[holiday]
                ,finctioning_day[functioning_day],str_to_date["day"],str_to_date["month"],str_to_date["year"]]

features_name = [ 'Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year']


df_u_input = pd.DataFrame([u_input_list],columns = features_name)
df_u_input


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,21,34,85,67,100,0.0,0.7,0.2,0,1,10,6,2023


In [122]:
u_input_list

[21, 34, 85, 67, 100, 0.0, 0.7, 0.2, 0, 1, 10, 6, 2023]

In [123]:
def season_to_df(seasons):
    seasons_cols = ["Spring","Summer","Winter"]

    seasons_data = np.zeros((1,len(seasons_cols)),dtype='int')

    df_seasons = pd.DataFrame(seasons_data,columns = seasons_cols)

    if seasons in seasons_cols:
        df_seasons[seasons]=1
    return df_seasons
    


    
df_seasons = season_to_df(seasons)

In [126]:
def days_to_df(week_day):
    days_names = ['Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday']
    days_name_data = np.zeros((1,len(days_names)),dtype="int")
    df_days = pd.DataFrame(days_name_data,columns = days_names)
    
    if week_day in days_names:
        df_days[week_day]=1
    return df_days


df_days = days_to_df(str_to_date["weekday"])

df_days
    

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [125]:
features_name

['Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)',
 'Holiday',
 'Functioning Day',
 'Day',
 'Month',
 'Year']

In [ ]:
['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year', 'Spring', 'Summer', 'Winter', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

In [128]:
df_for_pred = pd.concat([df_u_input,df_seasons,df_days],axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,21,34,85,67,100,0.0,0.7,0.2,0,1,...,2023,0,1,0,0,1,0,0,0,0


In [131]:
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 1.36681471,  1.76076456,  1.31239433, 63.11301355, -2.21005725,
        -0.65681529,  0.46358031,  0.29083855, -0.22754885,  0.18254726,
        -0.64636159, -0.14853965, 18.24263884, -0.57427456,  1.71961809,
        -0.57866842, -0.39840954,  2.45255275, -0.41815465, -0.40773843,
        -0.40559398, -0.41815465]])

In [135]:
val = model.predict(sc_data_for_pred)
val 

array([502.4063], dtype=float32)